### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[ 10.  10.  10.  10.]
 [ 10.   4.   9.  10.]
 [  8.   9.  10.   5.]
 [  9.   8.  10.  10.]
 [ 10.   5.   9.   9.]
 [  6.   4.  10.   6.]
 [  9.   8.  10.   9.]
 [ 10.   5.   9.   8.]
 [  7.   8.  10.   8.]
 [  9.   5.   9.   7.]
 [  9.   8.  10.   8.]
 [  9.  10.  10.   9.]
 [ 10.   9.  10.   8.]
 [  5.   8.   5.   8.]
 [ 10.   8.  10.  10.]
 [  9.   9.  10.  10.]
 [  9.   8.   8.   8.]
 [ 10.   8.   1.  10.]
 [  5.   6.  10.  10.]
 [  8.   7.  10.   7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [7]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = n_users * n_movies# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features)# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    y = np.dot(user_mat[i,:], movie_mat[:,j])
                    diff = ratings_mat[i, j] - y
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])
        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))

    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [8]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)# use your function with 4 latent features, lr of 0.005 and 10 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 49.597839
2 		 20.020146
3 		 5.336563
4 		 3.010776
5 		 2.785953
6 		 2.731561
7 		 2.698867
8 		 2.669721
9 		 2.639773
10 		 2.607448


In [9]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.11733206   8.89328058  10.57173381  10.1494556 ]
 [  8.69739588   7.41108489   9.02856173   8.82292922]
 [  7.776201     7.06545585   8.52186328   7.51127492]
 [  9.51126722   8.47328049  10.1990935    9.35023846]
 [  8.55103793   7.42301303   9.0845578    8.43332669]
 [  6.84344094   6.0152591    7.25398012   6.78810136]
 [  9.20772609   8.10394811   9.86231088   8.99279641]
 [  8.2277721    7.21847091   8.57148918   8.20301548]
 [  8.53218254   7.31845455   8.73188094   8.6756302 ]
 [  7.65423598   6.86975889   8.07588112   7.58798419]
 [  8.88887376   7.71187047   9.55604408   8.68045106]
 [  9.63665701   8.46172133  10.04729835   9.50733903]
 [  9.2492252    8.1990314    9.77256852   9.16416452]
 [  6.64023414   5.73992856   6.67721818   6.89821246]
 [  9.76089618   8.49408909  10.25184478   9.72955682]
 [  9.74084459   8.55098388  10.20417772   9.73569556]
 [  8.25621946   7.39048539   8.63802761   8.27879825]
 [  7.16267394   6.04059173   7.11392207   7.45204705]
 [  8.4705

**Write your summary here.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [10]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 42.508760
2 		 13.839475
3 		 3.746538
4 		 2.756569
5 		 2.677724
6 		 2.645560
7 		 2.618148
8 		 2.588999
9 		 2.556121
10 		 2.518625
11 		 2.475861
12 		 2.427250
13 		 2.372278
14 		 2.310549
15 		 2.241837
16 		 2.166155
17 		 2.083807
18 		 1.995434
19 		 1.902027
20 		 1.804900
21 		 1.705613
22 		 1.605859
23 		 1.507312
24 		 1.411483
25 		 1.319588
26 		 1.232471
27 		 1.150586
28 		 1.074030
29 		 1.002616
30 		 0.935972
31 		 0.873624
32 		 0.815072
33 		 0.759838
34 		 0.707503
35 		 0.657710
36 		 0.610178
37 		 0.564689
38 		 0.521088
39 		 0.479275
40 		 0.439198
41 		 0.400849
42 		 0.364253
43 		 0.329460
44 		 0.296539
45 		 0.265564
46 		 0.236606
47 		 0.209722
48 		 0.184950
49 		 0.162300
50 		 0.141754
51 		 0.123262
52 		 0.106747
53 		 0.092107
54 		 0.079220
55 		 0.067950
56 		 0.058153
57 		 0.049682
58 		 0.042391
59 		 0.036143
60 		 0.030804
61 		 0.026256
62 		 0.022390
63 		 0.019107
64 		

In [11]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 10.00000117  10.00000069  10.0000004    9.99999859]
 [  9.99999947   3.99999966   8.99999979  10.00000065]
 [  8.00000043   9.00000019  10.00000011   4.99999953]
 [  9.           7.99999999   9.99999998   9.99999999]
 [ 10.00000338   5.00000191   9.00000111   8.999996  ]
 [  5.99999791   3.99999872   9.99999923   6.00000252]
 [  8.99999739   7.99999841   9.99999906   9.00000315]
 [  9.99999924   4.99999958   8.99999976   8.00000089]
 [  6.9999995    7.99999975   9.99999986   8.00000058]
 [  9.00000027   5.0000002    9.00000012   6.99999966]
 [  9.00000105   8.00000063  10.00000038   7.99999875]
 [  9.00000041  10.00000024  10.00000014   8.99999951]
 [ 10.00000025   9.00000019  10.00000009   7.99999965]
 [  5.00000015   8.00000004   5.00000002   7.99999985]
 [  9.99999916   7.99999953   9.99999972  10.00000097]
 [  9.00000011   9.00000009  10.00000005   9.99999985]
 [  8.99999806   7.99999885   7.99999933   8.00000234]
 [  9.99999964   7.99999979   0.99999988  10.00000043]
 [  5.0000

**Write your summary here.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [12]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,  10.,  10.],
        [ 10.,   4.,   9.,  10.],
        [  8.,   9.,  10.,   5.],
        [  9.,   8.,  10.,  10.],
        [ 10.,   5.,   9.,   9.],
        [  6.,   4.,  10.,   6.],
        [  9.,   8.,  10.,   9.],
        [ 10.,   5.,   9.,   8.],
        [  7.,   8.,  10.,   8.],
        [  9.,   5.,   9.,   7.],
        [  9.,   8.,  10.,   8.],
        [  9.,  10.,  10.,   9.],
        [ 10.,   9.,  10.,   8.],
        [  5.,   8.,   5.,   8.],
        [ 10.,   8.,  10.,  10.],
        [  9.,   9.,  10.,  10.],
        [  9.,   8.,   8.,   8.],
        [ 10.,   8.,   1.,  10.],
        [  5.,   6.,  10.,  10.],
        [  8.,   7.,  10.,   7.]])

In [13]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)#use your function with 4 latent features, lr of 0.005 and 250 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 43.525219
2 		 15.285576
3 		 4.244459
4 		 2.727806
5 		 2.533042
6 		 2.452126
7 		 2.383037
8 		 2.311169
9 		 2.233550
10 		 2.149945
11 		 2.061102
12 		 1.968350
13 		 1.873431
14 		 1.778340
15 		 1.685109
16 		 1.595579
17 		 1.511177
18 		 1.432769
19 		 1.360606
20 		 1.294367
21 		 1.233288
22 		 1.176327
23 		 1.122337
24 		 1.070208
25 		 1.018981
26 		 0.967906
27 		 0.916476
28 		 0.864437
29 		 0.811764
30 		 0.758639
31 		 0.705411
32 		 0.652549
33 		 0.600589
34 		 0.550085
35 		 0.501559
36 		 0.455460
37 		 0.412136
38 		 0.371818
39 		 0.334620
40 		 0.300550
41 		 0.269525
42 		 0.241400
43 		 0.215985
44 		 0.193071
45 		 0.172441
46 		 0.153885
47 		 0.137208
48 		 0.122229
49 		 0.108786
50 		 0.096734
51 		 0.085940
52 		 0.076286
53 		 0.067664
54 		 0.059977
55 		 0.053133
56 		 0.047050
57 		 0.041651
58 		 0.036864
59 		 0.032626
60 		 0.028877
61 		 0.025563
62 		 0.022635
63 		 0.020050
64 		

In [14]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.013661632219824:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [15]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat =  FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)#fit to 1000 users with 4 latent features, lr of 0.005, and 20 iterations

Optimization Statistics
Iterations | Mean Squared Error 
1 		 0.008091
2 		 0.003751
3 		 0.002574
4 		 0.001986
5 		 0.001616
6 		 0.001358
7 		 0.001166
8 		 0.001017
9 		 0.000899
10 		 0.000803
11 		 0.000724
12 		 0.000658
13 		 0.000602
14 		 0.000554
15 		 0.000513
16 		 0.000478
17 		 0.000447
18 		 0.000421
19 		 0.000398
20 		 0.000377


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [16]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))# How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()


ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings# How many ratings did we make for user-movie pairs that didn't actually have ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")